In [29]:
import random
from typing import List, Tuple

def print_board(board):
    """Print the chess board in a formatted way."""
    for row in board:
        print(" ".join(str(cell) for cell in row))
    print()
    
def initial_board(n: int) -> List[List[int]]:
    """
    Create an initial n x n chess board for the N-Queens problem.
    
    Args:
        n (int): The number of queens.
        
    Returns:
        List[List[int]]: An empty chessboard matrix of size n x n, with default values set to 0.
    """
    board = [[0 for _ in range(n)] for _ in range(n)]
    return board

def random_queens(board: List[List[int]]) -> List[List[int]]:
    """
    Randomly place queens on the given chess board.

    Args:
        board (List[List[int]]): A chessboard matrix where queens will be placed.

    Returns:
        List[List[int]]: The chessboard matrix with queens randomly placed.
    """
    n = len(board)
    row_indices = list(range(n))
    random.shuffle(row_indices)  # Shuffle the row indices
    
    for row, col in enumerate(row_indices):
        board[row][col] = 'Q'
    
    return board

def is_conflict(board, row1, col1, row2, col2) -> bool:
    """
    Check if two queens are in conflict.

    Args:
        board (List[List[int]]): The chessboard matrix.
        row1 (int): Row index of the first queen.
        col1 (int): Column index of the first queen.
        row2 (int): Row index of the second queen.
        col2 (int): Column index of the second queen.

    Returns:
        bool: True if the queens are in conflict (same row, same column, or diagonal), False otherwise.
    """
    if board[row1][col1] == 'Q' and board[row2][col2] == 'Q':
        if row1 == row2 or col1 == col2:  # Same row or same column
            return True
        if abs(row1 - row2) == abs(col1 - col2):  # Diagonal conflict
            return True
    return False

def find_positions(board, element) -> List[Tuple[int, int]]:
    """
    Find positions of the specified element on the board.

    Args:
        board (List[List[int]]): The chessboard matrix.
        element: The element to search for.

    Returns:
        List[Tuple[int, int]]: A list of (row, column) tuples representing positions of the element.
    """
    positions = []
    for row_idx, row in enumerate(board):
        for col_idx, value in enumerate(row):
            if value == element:
                positions.append((row_idx, col_idx))
    return positions

def count_conflicts(board, queen_positions) -> int:
    """
    Count the total number of conflicts between queens on the board.

    Args:
        board (List[List[int]]): The chessboard matrix.
        queen_positions (List[Tuple[int, int]]): List of queen positions.

    Returns:
        int: The total number of conflicts between queens.
    """
    total_conflicts = 0
    for i in range(len(queen_positions)):
        for j in range(i + 1, len(queen_positions)):
            row1, col1 = queen_positions[i]
            row2, col2 = queen_positions[j]
            conflict = is_conflict(board, row1, col1, row2, col2)
            if conflict:
                total_conflicts += 1
    return total_conflicts

def evaluate_state(board) -> float:
    """
    Evaluate the state of the board by counting conflicts.

    Args:
        board (List[List[int]]): The chessboard matrix.

    Returns:
        float: The evaluation score indicating the quality of the board state.
    """
    return (len(board) - 1) * len(board) / 2 - count_conflicts(board, find_positions(board, 'Q'))

def move_queen(board: List[List[int]], row: int, new_row: int) -> None:
    """
    Move a queen to a different row within its column.

    Args:
        board (List[List[int]]): The chessboard matrix.
        row (int): Row index of the queen to be moved.
        new_row (int): New row index to move the queen to.
    """
    queen_col = board[row].index('Q')  # Find the column of the queen in the original row
    board[row][queen_col] = 0
    board[new_row][queen_col] = 'Q'

def generate_neighbors(board: List[List[int]]) -> List[List[int]]:
    """
    Generate neighboring configurations by moving queens to different rows.

    Args:
        board (List[List[int]]): The chessboard matrix.

    Returns:
        List[List[int]]: A list of neighboring chessboard matrices.
    """
    neighbors = []
    queen_positions = find_positions(board, 'Q')
    n = len(board)

    for queen_row, queen_col in queen_positions:
        for new_row in range(n):
            if new_row != queen_row:
                neighbor = [row[:] for row in board]  # Create a copy of the board
                move_queen(neighbor, queen_row, new_row)
                neighbors.append(neighbor)

    return neighbors

def find_best_neighbor(neighbors: List[List[int]]) -> Tuple[List[List[int]], float]:
    """
    Find the best neighbor among the generated neighbors based on evaluation.

    Args:
        neighbors (List[List[int]]): A list of neighboring chessboard matrices.

    Returns:
        Tuple[List[List[int]], float]: A tuple containing the best neighbor and its evaluation score.
    """
    best_neighbor = None
    best_evaluation = float("-inf")
    
    for neighbor in neighbors:
        evaluation = evaluate_state(neighbor)
        if evaluation > best_evaluation:
            best_evaluation = evaluation
            best_neighbor = neighbor
    
    return best_neighbor, best_evaluation

def hill_climbing(board: List[List[int]], max_iterations: int = 1000) -> Tuple[List[List[int]], float]:
    """
    Perform local search using hill climbing to find a better solution.

    Args:
        board (List[List[int]]): The chessboard matrix.
        max_iterations (int): Maximum number of iterations for the hill climbing algorithm.

    Returns:
        Tuple[List[List[int]], float]: A tuple containing the best solution and its evaluation score.
    """
    current_board = board
    current_evaluation = evaluate_state(current_board)
    
    for _ in range(max_iterations):
        neighbors = generate_neighbors(current_board)
        best_neighbor, neighbor_evaluation = find_best_neighbor(neighbors)
        
        if neighbor_evaluation > current_evaluation:
            current_board = best_neighbor
            current_evaluation = neighbor_evaluation
        else:
            break
    return current_board, current_evaluation


sizes = [6, 8, 10, 12]
# sizes = [6]
for n in sizes:

    board = initial_board(n)
    board_with_queens = random_queens(board)

    queen_positions = find_positions(board, 'Q')
    neighbors = generate_neighbors(board_with_queens)

    best_neighbor, neighbor_evaluation = find_best_neighbor(neighbors)
    best_board, best_evaluation =  hill_climbing(board_with_queens)

    print(f"\nBest Solution for {n} queens:")
    print_board(best_board)
    print("Evaluation:", best_evaluation)


Best Solution for 6 queens:
0 0 0 0 Q 0
0 Q 0 0 0 0
0 0 0 0 0 Q
Q 0 Q 0 0 0
0 0 0 0 0 0
0 0 0 Q 0 0

Evaluation: 14.0

Best Solution for 8 queens:
0 0 0 Q 0 0 0 0
0 0 0 0 0 0 0 Q
Q 0 0 0 0 0 0 0
0 0 0 0 Q 0 0 0
0 Q 0 0 0 0 0 0
0 0 0 0 0 Q 0 0
0 0 0 0 0 0 0 0
0 0 Q 0 0 0 Q 0

Evaluation: 27.0

Best Solution for 10 queens:
0 0 0 0 0 Q 0 0 0 0
0 0 0 0 0 0 0 0 0 Q
0 Q 0 0 0 0 0 0 0 0
0 0 0 0 Q 0 0 0 0 0
0 0 Q 0 0 0 0 0 0 0
0 0 0 0 0 0 0 Q 0 0
0 0 0 Q 0 0 0 0 0 0
Q 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 Q 0
0 0 0 0 0 0 Q 0 0 0

Evaluation: 43.0

Best Solution for 12 queens:
Q 0 0 0 0 0 0 0 0 0 0 0
0 0 0 Q 0 Q 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 Q 0 0
0 0 0 0 0 0 Q 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 Q 0
0 0 Q 0 0 0 0 0 0 0 0 0
0 0 0 0 Q 0 0 0 0 0 0 0
0 Q 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 Q 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 Q
0 0 0 0 0 0 0 0 Q 0 0 0

Evaluation: 64.0


In [24]:
def random_search(n: int, max_attempts: int = 10000) -> Tuple[List[List[int]], int]:
    """
    Perform random search to find a solution for the N-Queens problem.

    Args:
        n (int): The number of queens and the board size.
        max_attempts (int): Maximum number of attempts to find a valid solution.

    Returns:
        Tuple[List[List[int]], int]: A tuple containing the solution board and the number of attempts made.
    """
    attempts = 0
    while attempts < max_attempts:
        board = initial_board(n)
        board_with_queens = random_queens(board)
        conflicts = count_conflicts(board_with_queens, find_positions(board_with_queens, 'Q'))
        if conflicts == 0:
            return board_with_queens, attempts
        attempts += 1
        print("Board during execution:")
        print_board(board_with_queens)
    return [], max_attempts

# Example usage
# sizes = [6, 8, 10, 12]
sizes = [6]
for n in sizes:
    solution_board, attempts_made = random_search(n)

    if solution_board:
        print(f"Solution for size {n} found after", attempts_made, "attempts:")
        print_board(solution_board)
        evaluation = evaluate_state(solution_board)
        print("Evaluation:", evaluation)
    else:
        print(f"Solution for size {n} not found after", attempts_made, "attempts.")


Board during execution:
0 0 Q 0 0 0
0 0 0 0 Q 0
0 0 0 0 0 Q
0 0 0 Q 0 0
Q 0 0 0 0 0
0 Q 0 0 0 0

Board during execution:
0 Q 0 0 0 0
0 0 0 Q 0 0
0 0 0 0 0 Q
Q 0 0 0 0 0
0 0 0 0 Q 0
0 0 Q 0 0 0

Board during execution:
0 0 0 0 Q 0
0 0 0 Q 0 0
0 0 Q 0 0 0
Q 0 0 0 0 0
0 0 0 0 0 Q
0 Q 0 0 0 0

Board during execution:
0 Q 0 0 0 0
0 0 Q 0 0 0
0 0 0 Q 0 0
Q 0 0 0 0 0
0 0 0 0 Q 0
0 0 0 0 0 Q

Board during execution:
0 0 0 0 Q 0
Q 0 0 0 0 0
0 0 0 Q 0 0
0 0 0 0 0 Q
0 Q 0 0 0 0
0 0 Q 0 0 0

Board during execution:
0 0 0 Q 0 0
0 0 0 0 Q 0
0 Q 0 0 0 0
Q 0 0 0 0 0
0 0 0 0 0 Q
0 0 Q 0 0 0

Board during execution:
0 0 0 0 0 Q
Q 0 0 0 0 0
0 0 0 Q 0 0
0 0 0 0 Q 0
0 Q 0 0 0 0
0 0 Q 0 0 0

Board during execution:
0 0 0 0 0 Q
0 Q 0 0 0 0
0 0 0 0 Q 0
0 0 Q 0 0 0
0 0 0 Q 0 0
Q 0 0 0 0 0

Board during execution:
0 0 0 0 0 Q
0 0 0 0 Q 0
0 0 0 Q 0 0
Q 0 0 0 0 0
0 Q 0 0 0 0
0 0 Q 0 0 0

Board during execution:
0 Q 0 0 0 0
0 0 0 Q 0 0
0 0 0 0 0 Q
Q 0 0 0 0 0
0 0 0 0 Q 0
0 0 Q 0 0 0

Board during execution:
0 0 0 